# Import Libraries

In [3]:
import tensorflow as tf
assert float(tf.__version__[:3]) >= 2.3

import os
import numpy as np
import glob
import time
import matplotlib.pyplot as plt
import time
from tensorflow.keras.preprocessing import image

from tensorflow.keras.applications import resnet50 as model
# import ResNet50, preprocess_input, decode_predictions

# download data

In [4]:
!mkdir data

!wget  -O data/img0.JPG "https://d17fnq9dkz9hgj.cloudfront.net/breed-uploads/2018/08/siberian-husky-detail.jpg?bust=1535566590&width=630"
!wget  -O data/img1.JPG "https://www.hakaimagazine.com/wp-content/uploads/header-gulf-birds.jpg"
!wget  -O data/img2.JPG "https://www.artis.nl/media/filer_public_thumbnails/filer_public/00/f1/00f1b6db-fbed-4fef-9ab0-84e944ff11f8/chimpansee_amber_r_1920x1080.jpg__1920x1080_q85_subject_location-923%2C365_subsampling-2.jpg"
!wget  -O data/img3.JPG "https://www.familyhandyman.com/wp-content/uploads/2018/09/How-to-Avoid-Snakes-Slithering-Up-Your-Toilet-shutterstock_780480850.jpg"

mkdir: cannot create directory ‘data’: File exists
--2021-11-10 14:56:08--  https://d17fnq9dkz9hgj.cloudfront.net/breed-uploads/2018/08/siberian-husky-detail.jpg?bust=1535566590&width=630
Resolving d17fnq9dkz9hgj.cloudfront.net (d17fnq9dkz9hgj.cloudfront.net)... 13.225.112.108, 13.225.112.72, 13.225.112.139, ...
Connecting to d17fnq9dkz9hgj.cloudfront.net (d17fnq9dkz9hgj.cloudfront.net)|13.225.112.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24112 (24K) [image/jpeg]
Saving to: ‘data/img0.JPG’

data/img0.JPG       100%[===================>]  23.55K  --.-KB/s    in 0s      

2021-11-10 14:56:08 (49.5 MB/s) - ‘data/img0.JPG’ saved [24112/24112]

--2021-11-10 14:56:08--  https://www.hakaimagazine.com/wp-content/uploads/header-gulf-birds.jpg
Resolving www.hakaimagazine.com (www.hakaimagazine.com)... 23.185.0.4, 2620:12a:8000::4, 2620:12a:8001::4
Connecting to www.hakaimagazine.com (www.hakaimagazine.com)|23.185.0.4|:443... connected.
HTTP request sent, await

# Save resnet model

In [6]:
resnet = model.ResNet50(weights='imagenet')
resnet.save('resnet50_saved_model') 

/home/keti/workspace/tf_2.5.0/lib/python3.6/site-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


INFO:tensorflow:Assets written to: resnet50_saved_model/assets


# Convertng saved model to tflite model

In [7]:
# A generator that provides a representative dataset
def representative_data_gen():
    dataset_list = glob.glob('./data/*')
    for i in range(100):
        image = next(iter(dataset_list))
        image = tf.io.read_file(image)
        
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [224, 224])
        image = tf.cast(image / 255., tf.float32)
        image = tf.expand_dims(image, 0)
        yield [image]

# path to the SavedModel directory
converter = tf.lite.TFLiteConverter.from_saved_model('resnet50_saved_model') 

# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]  

 # This sets the representative dataset for quantization
converter.representative_dataset = representative_data_gen  

# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
converter.target_spec.supported_types = [tf.int8] 

# These set the input and output tensors to uint8 (added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

# Save the model.
with open('resnet50.tflite', 'wb') as f:
    f.write(tflite_model)

# Test converted model

## test base resnet50 tflite model

In [8]:
# interpreter = tf.lite.Interpreter('resnet50.tflite')

# interpreter.allocate_tensors()

In [19]:
import tflite_runtime.interpreter as tflite

start_time = time.time()
interpreter = tflite.Interpreter('resnet50.tflite' , 
                          experimental_delegates=[tflite.load_delegate('libedgetpu.so.1')])
interpreter.allocate_tensors()
end_time = time.time()
term = end_time - start_time
print(f'loading term: {term} sec')

loading term: 0.013535499572753906 sec


### Test speed

In [10]:
# 2. input 값을 설정

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
img_path = 'data/img0.JPG'  # Siberian_husky
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = model.preprocess_input(x)
x = tf.constant(x)

input_shape = input_details[0]['shape']
input_data = np.array(x, dtype=np.uint8)
interpreter.set_tensor(input_details[0]['index'], input_data)

In [13]:
total = 0

for i in range(0, 50):
    start_time = time.time()
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    end_time = time.time()
    term = end_time - start_time
    total += term
    print(f'term: {term} sec')

term: 1.15474534034729 sec
term: 1.1549546718597412 sec
term: 1.1556484699249268 sec
term: 1.1473743915557861 sec
term: 1.1647369861602783 sec
term: 1.1653625965118408 sec
term: 1.1625621318817139 sec
term: 1.1651318073272705 sec
term: 1.1617858409881592 sec
term: 1.1604323387145996 sec
term: 1.1641733646392822 sec
term: 1.1604223251342773 sec
term: 1.1629068851470947 sec
term: 1.1599714756011963 sec
term: 1.160404920578003 sec
term: 1.1607768535614014 sec
term: 1.160083293914795 sec
term: 1.1595497131347656 sec
term: 1.1654174327850342 sec
term: 1.1631524562835693 sec
term: 1.1665775775909424 sec
term: 1.1762974262237549 sec
term: 1.168868064880371 sec
term: 1.1666865348815918 sec
term: 1.1602075099945068 sec
term: 1.1664848327636719 sec
term: 1.1589365005493164 sec
term: 1.160688877105713 sec
term: 1.1618900299072266 sec
term: 1.1641952991485596 sec
term: 1.1584351062774658 sec
term: 1.159543514251709 sec
term: 1.1603832244873047 sec
term: 1.1591010093688965 sec
term: 1.1599843502044

In [14]:
total/50

1.1600894212722779

In [15]:
50/total

0.8620025160675057

## test compiled tflite model

`edgetpu_compiler` 를 실행하여 모델을 컴파일한다. 컴파일이 성공하면 뒤에 `_edgetpu` 가 붙은 `tflite` 모델이 생성된다. 
예시로 `resnet50.tflite` 파일을 컴파일하면 `resnet50_edgetpu.tflite` 가 생성된다


```bash
$ edgetpu_compiler resnet50.tflite
Edge TPU Compiler version 2.0.291256449

Model compiled successfully in 1618 ms.

Input model: resnet50.tflite
Input size: 25.08MiB
Output model: resnet50_edgetpu.tflite
Output size: 24.87MiB
On-chip memory available for caching model parameters: 6.25MiB
On-chip memory used for caching model parameters: 6.25MiB
Off-chip memory used for streaming uncached model parameters: 18.43MiB
Number of Edge TPU subgraphs: 1
Total number of operations: 77
Operation log: resnet50_edgetpu.log
See the operation log file for individual operation details.
```

In [20]:
import tflite_runtime.interpreter as tflite

start_time = time.time()
interpreter = tflite.Interpreter('resnet50_edgetpu.tflite' , 
                          experimental_delegates=[tflite.load_delegate('libedgetpu.so.1')])
interpreter.allocate_tensors()
end_time = time.time()
term = end_time - start_time
print(f'loading term: {term} sec')

loading term: 0.01430058479309082 sec


In [21]:
# 2. input 값을 설정

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
img_path = 'data/img0.JPG'  # Siberian_husky
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = model.preprocess_input(x)
x = tf.constant(x)

input_shape = input_details[0]['shape']
input_data = np.array(x, dtype=np.uint8)
interpreter.set_tensor(input_details[0]['index'], input_data)

In [22]:
total = 0
for i in range(0, 50):
    start_time = time.time()
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    end_time = time.time()
    term = end_time - start_time
    total += term
    print(f'term: {term} sec')

term: 0.057613372802734375 sec
term: 0.043219566345214844 sec
term: 0.04343247413635254 sec
term: 0.043280839920043945 sec
term: 0.04334235191345215 sec
term: 0.043236494064331055 sec
term: 0.043360233306884766 sec
term: 0.04332613945007324 sec
term: 0.04330158233642578 sec
term: 0.04323840141296387 sec
term: 0.04323911666870117 sec
term: 0.043271541595458984 sec
term: 0.04324054718017578 sec
term: 0.04318714141845703 sec
term: 0.04337024688720703 sec
term: 0.04326820373535156 sec
term: 0.04331851005554199 sec
term: 0.04381132125854492 sec
term: 0.04314708709716797 sec
term: 0.04319357872009277 sec
term: 0.043274641036987305 sec
term: 0.04324626922607422 sec
term: 0.04320406913757324 sec
term: 0.0432741641998291 sec
term: 0.0431513786315918 sec
term: 0.04321479797363281 sec
term: 0.043313026428222656 sec
term: 0.043331146240234375 sec
term: 0.04316282272338867 sec
term: 0.043243408203125 sec
term: 0.04321789741516113 sec
term: 0.04306316375732422 sec
term: 0.04318881034851074 sec
term:

In [23]:
total/50

0.04355556964874267

In [24]:
50/total

22.959176244613005